In [1]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as f
from pyspark.ml.fpm import FPGrowth

In [2]:
path = "C:/Users/Gui Mendes/Documents/archive/2017-07_bme280sof.csv"

In [3]:
spark = SparkSession\
    .builder\
    .appName("TP")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", "2")\
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [4]:
df = spark.read.csv(path, header='true',)

In [5]:
#df.show(10)
#df.schema

In [6]:
df = df.withColumn(
    'ocorrChuva',
    f.when((f.col("humidity") <= 60.00) , 0)\
    .otherwise(1))

df = df.withColumn("timestamp",f.to_timestamp(f.col("timestamp")))
df = df.withColumn("date",f.to_date(f.col("timestamp"),"yyyy-MM-dd"))
df = df.withColumn('mesAno',f.date_format(f.col('date'), '1/M/yyyy'))
#df.show(10)

In [48]:
#Média de chuvas por hora
dfOcorrChuvas = df\
    .groupBy(f.hour("timestamp").alias("hora"))\
    .agg(f.avg("ocorrChuva").alias("media"))

dfOcorrChuvas = dfOcorrChuvas.withColumn('período', f.when( (f.col('hora') >=0) & (f.col('hora') < 6) , "Madrugada").
                            otherwise( f.when(   (f.col('hora') >= 6) & (f.col('hora') < 12)  , "Manhã").
                            otherwise( f.when(   (f.col('hora') >=12) & (f.col('hora') < 18)  , "Tarde").
                            otherwise( f.when(   (f.col('hora') >= 18) & (f.col('hora') <= 24)  , "Noite").
                            otherwise('NULL')))))

In [49]:
dfOcorrChuvas.sort(dfOcorrChuvas.media.desc()).show(truncate=False)

+----+-------------------+---------+
|hora|media              |período  |
+----+-------------------+---------+
|3   |0.744144206483756  |Madrugada|
|2   |0.7265909872721018 |Madrugada|
|1   |0.6680240914379577 |Madrugada|
|0   |0.5979342787019362 |Madrugada|
|4   |0.5522310798219278 |Madrugada|
|23  |0.5035417725809185 |Noite    |
|22  |0.44352521721070304|Noite    |
|21  |0.3772652949993187 |Noite    |
|5   |0.35908715974704425|Madrugada|
|20  |0.3058365758754864 |Noite    |
|19  |0.26481770656527936|Noite    |
|18  |0.22077966216905143|Noite    |
|6   |0.21456733762191327|Manhã    |
|17  |0.17519738633269807|Tarde    |
|16  |0.17035699686605804|Tarde    |
|7   |0.13880061506919528|Manhã    |
|15  |0.10927231245956345|Tarde    |
|8   |0.08750384839051757|Manhã    |
|13  |0.08594633494397039|Tarde    |
|14  |0.07887391067538127|Tarde    |
+----+-------------------+---------+
only showing top 20 rows



In [55]:
""""dfOcorrChuvasCollect = df.where(f.col("ocorrChuva") == 1).groupBy(f.hour("timestamp")).agg(
  f.collect_list(f.col("humidity")).alias("chumidity")
,f.collect_list(f.col("pressure")).alias("cpressure") 
)
dfOcorrChuvasCollect.show(10)"""

'"dfOcorrChuvasCollect = df.where(f.col("ocorrChuva") == 1).groupBy(f.hour("timestamp")).agg(\n  f.collect_list(f.col("humidity")).alias("chumidity")\n,f.collect_list(f.col("pressure")).alias("cpressure") \n)\ndfOcorrChuvasCollect.show(10)'

In [56]:
""""dfMedias = df\
    .groupBy(f.hour("timestamp").alias("hora"))\
    .agg(f.avg("humidity").alias("mediaH"),\
    f.avg("pressure").alias("mediaP"),\
    f.avg("temperature").alias("mediaT"),\
    f.avg("ocorrChuva").alias("mediaChuvas"))

dfMedias.show()"""

'"dfMedias = df    .groupBy(f.hour("timestamp").alias("hora"))    .agg(f.avg("humidity").alias("mediaH"),    f.avg("pressure").alias("mediaP"),    f.avg("temperature").alias("mediaT"),    f.avg("ocorrChuva").alias("mediaChuvas"))\n\ndfMedias.show()'